# Snake Tutorial Solutions

shhh don't tell the kids

In [7]:
# Import statements

from tkinter import Tk, Canvas, Frame, BOTH
from random import randint, choice

In [8]:
class Block:
    def __init__(self, x, y, size=20):
        self.x = x
        self.y = y
        self.size = size
    
    def draw(self, canvas, color):
        x1 = self.x * self.size
        y1 = self.y * self.size
        x2 = x1 + self.size
        y2 = y1 + self.size
        canvas.create_rectangle(x1, y1, x2, y2, outline='', fill=color)
    
    def equals(self, other):
        return self.x == other.x and self.y == other.y

In [11]:
class Game:

    def __init__(self):
        self.score = 0
        self.width = 20
        self.height = 15
        self.block_size = 20
        self.snake = Snake(self.width // 2, self.height // 2)
        self.food = Food(self.snake.blocks, self.width, self.height)
        
        self.create_canvas(self.width, self.height, self.block_size)
    
    # Already completed
    def create_canvas(self, width, height, size):
        def on_press(event):
            key = event.keysym
            if key in ['Left', 'Right', 'Up', 'Down']:
                self.snake.move(key)
            elif key == 'q':
                window.destroy()
        
        self.window = Tk()
        self.window.geometry('{}x{}'.format(width * size, height * size))
        self.window.resizable(False, False)
        self.window.bind('<Key>', on_press)

        frame = Frame(self.window)
        frame.master.title('Snake')
        frame.pack(fill=BOTH, expand=1)

        self.canvas = Canvas(frame)
        self.canvas.pack(fill=BOTH, expand=1)
        
        
    def start(self):
        self.tick()
        self.window.mainloop()
    
    def crashed(self, snake):
        head = snake.head()
        for b in snake.blocks[1:]:
            if head.equals(b):
                return True
        in_bounds = 0 <= head.x < self.width and 0 <= head.y < self.height
        return not in_bounds
    
    def tick(self):
        ate_food = self.food.eaten(self.snake)
        self.snake.update(ate_food)
        if ate_food:
            self.food = Food(self.snake.blocks, self.width, self.height)
            self.score += 1
            # Uncomment below if game speed increases after every food eaten
            # self.snake.speed += 1
        self.render()
        if self.crashed(self.snake):
            print(f'Snake died with score {self.score}.')
            self.window.destroy()
        self.window.after(int(1000 / self.snake.speed), self.tick)
        
    def render(self):
        self.canvas.delete('all')
        self.snake.draw(self.canvas)
        self.food.draw(self.canvas)

In [10]:
class Food:
    def __init__(self, invalid_blocks, width, height, size=20):
        self.block = Block(randint(0, width - 1), randint(0, height - 1), size)
        while not self.is_valid(invalid_blocks):
            self.block = Block(randint(0, width - 1), randint(0, height - 1), size)
        
    def is_valid(self, invalid_blocks):
        for b in invalid_blocks:
            if self.block.equals(b):
                return False
        return True
        
    def draw(self, canvas):
        self.block.draw(canvas, 'red')
        
    def eaten(self, snake):
        return self.block.equals(snake.head())

In [13]:
class Snake:
    
    start_length = 4
    start_speed = 7
    
    def __init__(self, startX, startY):
        self.blocks = []
        for i in range(Snake.start_length):
            self.blocks.append(Block(startX - i, startY))
        self.direction = 'Right'
        self.speed = Snake.start_speed
        self.paused = True
    
    def move(self, direction):
        if direction == 'Left' and self.direction != 'Right' or \
           direction == 'Right' and self.direction != 'Left' or \
           direction == 'Up' and self.direction != 'Down' or \
           direction == 'Down' and self.direction != 'Up':
            self.direction = direction
            self.paused = False
    
    def head(self):
        return self.blocks[0]

    def update(self, ate_food):
        if self.paused:
            return
        
        x = self.head().x
        y = self.head().y
        
        if self.direction == 'Left':
            next_block = Block(x - 1, y)
        elif self.direction == 'Right':
            next_block = Block(x + 1, y)
        elif self.direction == 'Up':
            next_block = Block(x, y - 1)
        elif self.direction == 'Down':
            next_block = Block(x, y + 1)
        else:
            return
        
        self.blocks.insert(0, next_block)

        if not ate_food:
            self.blocks.pop()
    
    def draw(self, canvas):
        for block in self.blocks:
            block.draw(canvas, 'grey')

In [14]:
game = Game()
game.start()

Snake died with score 6.


In [40]:
def test_snake():
    snake = Snake(0, 0)
    
    opposites = {
        'Left': 'Right',
        'Right': 'Left',
        'Up': 'Down',
        'Down': 'Up'
    }

    positions = [(0, -1), (-1, -1), (-1, 0), (0, 0), (0, 1), (0, 2), (1, 2)]
    directions = ['Up', 'Left', 'Down', 'Right', 'Down', 'Down', 'Right']

    def test_snake_direction(direction, ate_food, position, move=True):
        nonlocal current_length
        if move:
            snake.move(direction)
            assert snake.direction == direction, f'Snake.move is incorrect when moving {direction} :('
        snake.move(opposites[direction])
        assert snake.direction == direction, f'Snake.move is incorrect when moving {direction} :('
        snake.update(ate_food)
        if ate_food:
            current_length += 1
        assert len(snake.blocks) == current_length, f'Snake.update is incorrect when moving {direction} :('
        x, y = position
        assert snake.head().x == x and snake.head().y == y, f'Snake.update is incorrect when moving {direction} :('

    current_length = 4
    test_snake_direction('Right', False, (0, 0), False)

    for position, direction in zip(positions, directions):
        test_snake_direction(direction, choice([True, False]), position, True)

test_snake()

In [46]:
def test_food():
    width = 1000
    height = 2000
    size = 50
    invalid_blocks = [Block(randint(0, width - 1), randint(0, height - 1), size) for _ in range(3500)]
    food = Food(invalid_blocks, width, height)
    for b in invalid_blocks:
        assert food.block.x != b.x or food.block.y != b.y

test_food()